In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
google_api_key=os.getenv("GEMINI_API_KEY") #to get key

In [ ]:
if google_api_key=="":
    print("API key not found")
else:
    print("API key found")

In [ ]:

from llama_index.llms.gemini import Gemini

import google.generativeai as genai

from llama_index.core import SimpleDirectoryReader #used to load data

from llama_index.core import VectorStoreIndex

from IPython.display import Markdown, display

from llama_index.core import ServiceContext

from llama_index.core import StorageContext, load_index_from_storage

from llama_index.embeddings.gemini import GeminiEmbedding

In [ ]:
genai.configure(api_key=google_api_key) #to check if api is working or not

In [ ]:
#checing if i can call my models or not using the key
for models in genai.list_models():
    print(models)


In [ ]:
for models in genai.list_models():
  if 'generateContent' in models.supported_generation_methods:
    print(models.name)
#some of these models are text- to- text generation and image-to-text generation

In [ ]:
documents=SimpleDirectoryReader("../Data")

In [ ]:
doc=documents.load_data()

In [ ]:
print(doc[0].text)

In [ ]:
model=Gemini(models='gemini-pro',api_key=google_api_key) #main model is gemini-pro and under that we are using embedding-001

In [ ]:
gemini_embed_model=GeminiEmbedding(model_name="models/embedding-001")

In [ ]:
# # Configure Service Context- its a container that contains the following objects-index, query,helps in retrieving based on similar context(helps in indexing the embedding)
# import llama_index
# from llama_index.core import VectorStoreIndex
# from llama_index.llms.ollama import Ollama
# from llama_index.embeddings.gemini import GeminiEmbedding
# from llama_index.core import Settings
# from llama_index.core.node_parser import SentenceSplitter

# # Assuming you've already defined this:
# gemini_embed_model = GeminiEmbedding(model_name="models/embedding-001")

# # Configure LLM, embedding model, and node parser globally
# Settings.llm = Ollama(model="llama2", request_timeout=120.0)
# Settings.embed_model = gemini_embed_model
# Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
# index = VectorStoreIndex.from_documents(doc)

# Configure Service Context using Settings
Settings.llm = model
Settings.embed_model = gemini_embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

# Build index and query
index = VectorStoreIndex.from_documents(doc)
index.storage_context.persist()
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("what is a FLAIR?")
print(response)

In [ ]:

response = query_engine.query("what is the command for install the FLAIR library?")
print(response)

In [ ]:


response = query_engine.query("what is the proposed solution from this paper?")
print(response)

In [ ]:

response = query_engine.query("what github links of this framework?")
print(response)

In [ ]:

response = query_engine.query("what is the minimum python version to setup FLAIR in the environment?")
print(response)

In [ ]:

response = query_engine.query("What are the presented framework used for?")
print(response)